In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive/', force_remount = True)
os.chdir("/content/drive/MyDrive/CS470-Project1/")


Mounted at /content/drive/


In [ ]:
!git clone https://github.com/scaelles/DEXTR-PyTorch.git

fatal: destination path 'DEXTR-PyTorch' already exists and is not an empty directory.


In [ ]:
os.chdir("/content/drive/MyDrive/CS470-Project1/DEXTR-PyTorch/")

import os.path

from torch.utils.data import DataLoader
from evaluation.eval import eval_one_result
import dataloaders.pascal as pascal

In [ ]:
os.chdir("/content/drive/MyDrive/CS470-Project1/DEXTR-PyTorch/")
!git pull

Already up to date.


In [ ]:
!pip install tensorboardX

In [ ]:
os.chdir("/content/drive/MyDrive/CS470-Project1/DEXTR-PyTorch/")
import socket
import timeit
from datetime import datetime
import scipy.misc as sm
from collections import OrderedDict
import glob
import numpy as np
import imageio

# PyTorch includes
import torch
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.nn.functional import upsample

# Tensorboard include
from tensorboardX import SummaryWriter

# Custom includes
from dataloaders.combine_dbs import CombineDBs as combine_dbs
import dataloaders.pascal as pascal
import dataloaders.sbd as sbd
from dataloaders import custom_transforms as tr
import networks.deeplab_resnet as resnet
from layers.loss import class_balanced_cross_entropy_loss
from dataloaders.helpers import *



In [ ]:
# Set gpu_id to -1 to run in CPU mode, otherwise set the id of the corresponding gpu
gpu_id = 0
device = torch.device("cuda:"+str(gpu_id) if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print('Using GPU: {} '.format(gpu_id))

# Setting parameters
use_sbd = False
nEpochs = 100  # Number of epochs for training
resume_epoch = 0  # Default is 0, change if want to resume

p = OrderedDict()  # Parameters to include in report
classifier = 'psp'  # Head classifier to use
testBatch = 5  # Testing batch size
useTest = 1  # See evolution of the test set when training?
nTestInterval = 10  # Run on test set every nTestInterval epochs
snapshot = 20  # Store a model every snapshot epochs
relax_crop = 50  # Enlarge the bounding box by relax_crop pixels
nInputChannels = 4  # Number of input channels (RGB + heatmap of extreme points)
zero_pad_crop = True  # Insert zero padding when cropping the image


Using GPU: 0 


In [ ]:
# Network definition
modelName = 'dextr_pascal'

net = resnet.resnet101(1, pretrained=False, nInputChannels=nInputChannels, classifier=classifier)


print("Initializing the network train using DEXTR")

path_model = "/content/drive/MyDrive/CS470-Project1/DEXTR-PyTorch/models/dextr_pascal-sbd.pth"

net.load_state_dict(
        torch.load( path_model,
                   map_location=lambda storage, loc: storage))

net.to(device)



Constructing ResNet model...
Dilations: (2, 4)
Number of classes: 1
Number of Input Channels: 4
Initializing classifier: PSP
Initializing the network train using DEXTR


ResNet(
  (conv1): Conv2d(4, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
run_id = 1
save_dir_root = f'/content/drive/MyDrive/CS470-Project1/number_of_points/'
exp_name = "number_of_points"


if resume_epoch == 0:
    runs = sorted(glob.glob(os.path.join(save_dir_root, 'run_*')))
    run_id = int(runs[-1].split('_')[-1]) + 1 if runs else 0
else:
    run_id = 0
save_dir = os.path.join(save_dir_root, 'run_' + str(run_id))
if not os.path.exists(os.path.join(save_dir, 'models')):
    os.makedirs(os.path.join(save_dir, 'models'))


In [ ]:
net.eval()
composed_transforms_ts = transforms.Compose([
    tr.CropFromMask(crop_elems=('image', 'gt'), relax=relax_crop, zero_pad=zero_pad_crop),
    tr.FixedResize(resolutions={'gt': None, 'crop_image': (512, 512), 'crop_gt': (512, 512)}),
    tr.ExtremePoints(sigma=10, pert=0, elem='crop_gt'),
    tr.ToImage(norm_elem='extreme_points'),
    tr.ConcatInputs(elems=('crop_image', 'extreme_points')),
    tr.ToTensor()])
db_test = pascal.VOCSegmentation(split='val', transform=composed_transforms_ts, retname=True)
testloader = DataLoader(db_test, batch_size=1, shuffle=False, num_workers=1)

save_dir_res = os.path.join(save_dir, 'Results')
if not os.path.exists(save_dir_res):
    os.makedirs(save_dir_res)


AttributeError: ignored

In [ ]:
print('Testing Network')
with torch.no_grad():
    # Main Testing Loop
    for ii, sample_batched in enumerate(testloader):

        inputs, gts, metas = sample_batched['concat'], sample_batched['gt'], sample_batched['meta']

        # Forward of the mini-batch
        inputs = inputs.to(device)

        outputs = net.forward(inputs)
        outputs = upsample(outputs, size=(512, 512), mode='bilinear', align_corners=True)
        outputs = outputs.to(torch.device('cpu'))

        for jj in range(int(inputs.size()[0])):
            pred = np.transpose(outputs.data.numpy()[jj, :, :, :], (1, 2, 0))
            pred = 1 / (1 + np.exp(-pred))
            pred = np.squeeze(pred)
            gt = tens2image(gts[jj, :, :, :])
            bbox = get_bbox(gt, pad=relax_crop, zero_pad=zero_pad_crop)
            result = crop2fullmask(pred, bbox, gt, zero_pad=zero_pad_crop, relax=relax_crop)

            # Save the result, attention to the index jj
            
            imageio.imwrite(os.path.join(save_dir_res, metas['image'][jj] + '-' + metas['object'][jj] + '.png'), result)
            # sm.imsave(os.path.join(save_dir_res, metas['image'][jj] + '-' + metas['object'][jj] + '.png'), result)


Testing Network


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3509: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: overflow encountered in exp


In [ ]:
%cd /content/drive/MyDrive/CS470-Project1/DEXTR-PyTorch/
!python3 eval_all.py

/content/drive/MyDrive/CS470-Project1/DEXTR-PyTorch
tcmalloc: large alloc 1999642624 bytes == 0x55dd544a0000 @  0x7fca43d0c1e7 0x55dd4da02338 0x55dd4d9ccad7 0x55dd4db4b575 0x55dd4dae58b8 0x55dd4d9d03a2 0x55dd4daaec6e 0x55dd4d9cfff9 0x55dd4dac1d4d 0x55dd4da43ec8 0x55dd4d9d17aa 0x55dd4da43d30 0x55dd4da3ecdd 0x55dd4d9d188a 0x55dd4da43d30 0x55dd4da3ea2e 0x55dd4d9d213c 0x55dd4da13239 0x55dd4da10184 0x55dd4d9d09f9 0x55dd4da44937 0x55dd4da3ea2e 0x55dd4da3e723 0x55dd4db08812 0x55dd4db08b8d 0x55dd4db08a36 0x55dd4dae0183 0x55dd4dadfe2c 0x7fca42af6c87 0x55dd4dadfd0a
Files already downloaded and verified
tcmalloc: large alloc 1999642624 bytes == 0x55dd544a0000 @  0x7fca43d0c1e7 0x55dd4da02338 0x55dd4d9ccad7 0x55dd4db4b575 0x55dd4dae58b8 0x55dd4d9d03a2 0x55dd4daaec6e 0x55dd4d9cfff9 0x55dd4dac1d4d 0x55dd4da43ec8 0x55dd4d9d17aa 0x55dd4da43d30 0x55dd4da3ea2e 0x55dd4d9d213c 0x55dd4da13239 0x55dd4da10184 0x55dd4d9d09f9 0x55dd4da44937 0x55dd4da3ea2e 0x55dd4da3e723 0x55dd4db08812 0x55dd4db08b8d 0x55dd4db0